# Patch

You can change the behavior of an existing function or method by patching it with `unittest.mock.patch`. You just need to specify the `target`, which refers to an object in Python. Find out more in the [specific documentation page](https://docs.python.org/3/library/unittest.mock.html#unittest.mock.patch).

In [1]:
import requests

import unittest
from unittest.mock import patch

from sklearn.neighbors import KNeighborsRegressor

## Target

There’s always some mystery associated with specifying the `target` for mocking; this section focuses on the details.

Target should be a string in the form 'package.module.ClassName'. The target is imported and the specified object replaced with the new object, so the target must be importable from the environment you are calling `patch()` from. The target is imported when the decorated function is executed, not at decoration time.

**Note**: All targets in these notebook examples begin with the `__main__` section. This allows the patched object to be referenced in the same module as the patching code.

---

The following example shows really typical case. For testing code that handles API reponse, but it's typical not ot have access to the API during code development - mocking output of the requesting tool is good option in such case.

In [10]:
def request_user(user_id):
    response = requests.get(f"https://im_not_exist/{user_id}")
    if response.ok:
        return response.text
    else:
        return "Fail!"

Patching `requests.get` to return an object with the attribute `ok=True` allows us to ensure that we receive a response in a successful case.

In [ ]:
with patch("__main__.requests.get") as mocked_get:    
    mocked_get.return_value.ok = True
    mocked_get.return_value.text = "Success"
    print(request_user("User"))

Success


Conversely, simulating an error response case returns the message corresponding to a failure.

In [12]:
with patch("__main__.requests.get") as mocked_get:
    mocked_get.return_value.ok = False
    mocked_get.return_value.text = "Success"
    print(request_user("User"))

Fail!


### Class method

You can modify the behavior of a class method as well, simply by specifying the path to it after the class name.

---

As an example, consider another common case in my practice. When writing code that needs to handle a machine learning model, you don't necessarily need to use a specific model. Mocking the model's results is a good way to verify if everything works correctly. The following cell shows how to define the result of the `predict` method of the `__main__.KNeighboursRegressor` class.

In [9]:
with patch("__main__.KNeighborsRegressor.predict") as predict:
    regressor = KNeighborsRegressor()
    predict.return_value = 'predict out'
    print(regressor.predict())

predict out
